In [1]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.models as models
import tensorflow.keras.callbacks as callbacks
import tensorflow.keras.preprocessing.image as image
import tensorflow.keras.optimizers as optimizers
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
from imgaug import augmenters as iaa
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
train_labels = pd.read_csv('train_labels.csv',dtype= {'Id': 'str', 'Category': 'str'})

In [3]:
IMAGE_SIZE = 224

In [4]:
X = []
y = []
for i in range(train_labels.shape[0]):
    img = image.load_img('train/{}.jpg'.format(train_labels.iloc[i]['Id']),
                                                target_size=(IMAGE_SIZE, IMAGE_SIZE))
    X.append(image.img_to_array(img)/255.0)
    y.append(train_labels.iloc[i]['Category'])

In [5]:
X = np.asarray(X)

In [6]:
from tensorflow.keras.utils import to_categorical
classes = {'rose': 0, 'sunflower': 1, 'daisy': 2, 'tulip': 3, 'dandelion': 4}
yi = [classes[s] for s in y]
y_sparse = to_categorical(yi)

In [7]:
train_X, val_X, train_Y, val_Y = train_test_split(X, y_sparse, test_size=0.1, random_state=17, stratify=y_sparse)

In [51]:
base_model = tf.keras.applications.NASNetLarge(weights='imagenet', include_top=False)

/home/freefly19/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [75]:
x = base_model.output
x = layers.SpatialDropout2D(0.2)(x)
x = layers.GlobalAvgPool2D()(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.2)(x)
predictions = layers.Dense(len(classes), activation='softmax')(x)

m = models.Model(inputs=base_model.input, outputs=predictions)

In [53]:
class AugmentedSequence(utils.Sequence):
  def __init__(self, X, y, batch_size):
    self.X = np.array(X)
    self.y = np.array(y)
    self.batch_size = batch_size
    
    #for shuffling
    self.ids = np.random.permutation(range(len(X)))
    
    #for augmentation
    self.seq = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Flipud(0.2),
        iaa.Affine(scale=(0.8, 1.2), translate_percent=(-0.2, 0.2), rotate=(-45, 45), shear=(-16, 16))
    ])
    
  def __len__(self):
    return int(np.ceil(len(self.X)/float(self.batch_size)))
  
  def __getitem__(self, index):
    start = index * self.batch_size
    end = np.minimum((index + 1) * self.batch_size, len(self.X))
    
    ids = self.ids[start:end]
    
    batchX = self.X[ids]
    batchy = self.y[ids]
    
    batchX = self.seq.augment_images(batchX)
    
    return np.array(batchX), np.array(batchy)
  
  def on_epoch_end(self):    
    self.ids = np.random.permutation(range(len(self.X)))

In [73]:
base_model.trainable = False
m.compile(optimizer=optimizers.Adam(1e-6),
          loss='categorical_crossentropy',
          metrics=['accuracy'])
m.fit_generator(AugmentedSequence(train_X, train_Y, 8), 
      validation_data=(val_X, val_Y),
      epochs = 20)

Epoch 1/20
341/341 [==============================] - 71s 208ms/step - loss: 0.1301 - acc: 0.9520 - val_loss: 0.2203 - val_acc: 0.9406
Epoch 2/20
341/341 [==============================] - 35s 104ms/step - loss: 0.1009 - acc: 0.9644 - val_loss: 0.2161 - val_acc: 0.9439
Epoch 3/20
341/341 [==============================] - 35s 104ms/step - loss: 0.1169 - acc: 0.9597 - val_loss: 0.2310 - val_acc: 0.9439
Epoch 4/20
341/341 [==============================] - 35s 103ms/step - loss: 0.1095 - acc: 0.9569 - val_loss: 0.2287 - val_acc: 0.9439
Epoch 5/20
341/341 [==============================] - 35s 104ms/step - loss: 0.1026 - acc: 0.9674 - val_loss: 0.2316 - val_acc: 0.9439
Epoch 6/20
341/341 [==============================] - 35s 104ms/step - loss: 0.1173 - acc: 0.9561 - val_loss: 0.2383 - val_acc: 0.9439
Epoch 7/20
341/341 [==============================] - 35s 104ms/step - loss: 0.1100 - acc: 0.9609 - val_loss: 0.2427 - val_acc: 0.9472
Epoch 8/20
341/341 [==============================] - 3

In [76]:
base_model.trainable = True
m.compile(optimizer=optimizers.Adam(1e-6),
          loss='categorical_crossentropy',
          metrics=['accuracy'])
m.fit_generator(AugmentedSequence(train_X, train_Y, 8), 
      validation_data=(val_X, val_Y),
      epochs = 100,
      callbacks=[
          callbacks.ModelCheckpoint('weights-progress.h5', verbose=1, save_best_only=True, save_weights_only=True)
      ])

Epoch 1/100
  1/341 [..............................] - ETA: 3:51:24 - loss: 1.3142 - acc: 0.5000

ResourceExhaustedError: OOM when allocating tensor with shape[512,512,3,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node res5b_branch2b_1/Conv2D}} = Conv2D[T=DT_FLOAT, _class=["loc:@train...kpropInput"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](activation_92/Relu, res5b_branch2b_1/Conv2D/ReadVariableOp)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_14/mul/_27343}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_20485_loss_14/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of a

In [ ]:
base_model.trainable = True
m.compile(optimizer=optimizers.Adam(1e-8),
          loss='categorical_crossentropy',
          metrics=['accuracy'])
m.fit_generator(AugmentedSequence(np.concatenate((train_X, val_X)), np.concatenate((train_Y, val_Y)), 8), 
      validation_data=(val_X, val_Y),
      epochs = 20
)

In [ ]:
sub_num=5

In [ ]:
m.save_weights(f'weights-submission-{sub_num}.h5')

In [ ]:
import os
test_X = []
test_lebels = []
for img_file in os.listdir('test'):
    img = image.load_img('test/' + img_file, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    test_X.append(image.img_to_array(img)/255.0)
    test_lebels.append(img_file.replace('.jpg', ''))
test_X = np.asarray(test_X)

In [ ]:
test_pred = m.predict(test_X)

In [ ]:
class_indices = tf.argmax(test_pred, axis=1).eval(session=tf.Session())
reverted_classes = {classes[c]:c for c in classes}
pred_classes = [reverted_classes[c] for c in class_indices]

In [ ]:
df = pd.DataFrame(data={'Category': pred_classes}, index=test_lebels)

In [ ]:
df.index.name = 'Id'
df.sort_index().to_csv(f'res-{sub_num}.csv')